In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import warnings
import time
import torch
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score,  mean_absolute_error
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from sklearn.metrics import precision_score, average_precision_score, log_loss

In [5]:
# Methods
from sklearn.linear_model import LinearRegression

# Model selection and evaluation tools
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from skopt.space import Real, Categorical, Integer
from skopt import BayesSearchCV
from sklearn.metrics import mean_squared_error, r2_score,  mean_absolute_error

In [6]:
# Train data
data = pd.read_csv('C:/Users/susum/Documents/year5_sem1/QBUS3820/Group_Assignment/final_version/data_train.csv')
pd.set_option('display.max_columns', len(data.columns))
data.head()

,host_since,host_response_rate,host_is_superhost,host_listings_count,host_identity_verified,zipcode,latitude,longitude,is_location_exact,accommodates,bathrooms,bedrooms,security_deposit_perc,cleaning_fee_perc,guests_included,extra_people_perc,minimum_nights,maximum_nights,availability_90,number_of_reviews,first_review,last_review,review_scores_rating,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,reviews_per_month,price,flexible,moderate,Ashfield,Auburn,Bankstown,Blacktown,Botany_Bay,Burwood,Camden,Campbelltown,Canada_Bay,Canterbury,City_Of_Kogarah,Fairfield,Holroyd,Hornsby,Hunters_Hill,Hurstville,Ku_Ring_Gai,Lane_Cove,Leichhardt,Liverpool,Manly,Marrickville,Mosman,North_Sydney,Parramatta,Penrith,Pittwater,Randwick,Rockdale,Ryde,Strathfield,Sutherland_Shire,Sydney,The_Hills_Shire,Warringah,Waverley,Willoughby,Woollahra,Aparthotel,Apartment,Bed_and_breakfast,Boat,Boutique_hotel,Bungalow,Cabin,Camper_RV,Condominium,Cottage,Earth_house,Farm_stay,Guest_suite,Guesthouse,Hostel,Hotel,House,Island,Loft,Other,Serviced_apartment,Tiny_house,Townhouse,Villa,Yurt,Entire_home_apt,Hotel_room,Private_room,Shared_room,Couch,Futon,Pull_out_Sofa,Real_Bed,Missing,a_few_days_or_more,within_a_day,within_a_few_hours,within_an_hour,sent_id_dummy,sesame_dummy,sesame_offline_dummy,weibo_dummy,kba_dummy,zhima_selfie_dummy,manual_online_dummy,manual_offline_dummy,google_dummy,work_email_dummy,facebook_dummy,identity_manual_dummy,selfie_dummy,offline_government_id_dummy,jumio_dummy,reviews_dummy,government_id_dummy,email_dummy,phone_dummy,Washer_Dryer_dummy,Breakfast_bar_dummy,Infinity_pool_dummy,Parking_dummy,Projector_and_screen_dummy,Bidet_dummy,Shared_gym_dummy,Touchless_faucets_dummy,Pool_cover_dummy,Garage_parking_dummy,Private_gym_dummy,Bathtub_with_bath_chair_dummy,Brick_oven_dummy,Gated_property_dummy,Propane_barbeque_dummy,Apple_TV_dummy,Security_system_dummy,Warming_drawer_dummy,Stand_alone_steam_shower_dummy,Mountain_view_dummy,Ironing_Board_dummy,Ice_Machine_dummy,Gas_fireplace_dummy,Outdoor_kitchen_dummy,Jetted_tub_dummy,Steam_oven_dummy,Tennis_court_dummy,Amazon_Echo_dummy,HBO_GO_dummy,Shared_hot_tub_dummy,Ground_floor_access_dummy,Chef_s_kitchen_dummy,Dining_area_dummy,Fire_pit_dummy,Hammock_dummy,Sauna_dummy,Standing_valet_dummy,Fixed_grab_bars_for_toilet_dummy,Private_hot_tub_dummy,Heat_lamps_dummy,Private_bathroom_dummy,Murphy_bed_dummy,Exercise_equipment_dummy,Pool_toys_dummy,Shower_chair_dummy,Alfresco_bathtub_dummy,High_resolution_computer_monitor_dummy,Wine_cooler_dummy,Ski_in_Ski_out_dummy,Fixed_grab_bars_for_shower_dummy,Shared_pool_dummy,Firm_mattress_dummy,Double_oven_dummy,Private_pool_dummy,Heated_floors_dummy,Printer_dummy,Beach_view_dummy,Central_air_conditioning_dummy,Outdoor_parking_dummy,Day_bed_dummy,Gas_oven_dummy,Memory_foam_mattress_dummy,Mini_fridge_dummy,Sun_loungers_dummy,Wide_clearance_to_shower_dummy,_toilet_dummy,Heated_towel_rack_dummy,Step_free_shower_dummy,Wide_doorway_to_guest_bathroom_dummy,Soaking_tub_dummy,Formal_dining_area_dummy,Accessible_height_toilet_dummy,Table_corner_guards_dummy,Disabled_parking_spot_dummy,Wide_entrance_dummy,Other_pet_s__dummy,Wide_entryway_dummy,EV_charger_dummy,Accessible_height_bed_dummy,Kitchenette_dummy,DVD_player_dummy,Rain_shower_dummy,Terrace_dummy,Ceiling_fan_dummy,Pillow_top_mattress_dummy,Fireplace_guards_dummy,Extra_space_around_bed_dummy,En_suite_bathroom_dummy,Convection_oven_dummy,Sound_system_dummy,Breakfast_table_dummy,Bread_maker_dummy,Baby_monitor_dummy,Balcony_dummy,Doorman_dummy,Outdoor_seating_dummy,Netflix_dummy,Espresso_machine_dummy,Smart_TV_dummy,Walk_in_shower_dummy,Hot_water_kettle_dummy,Lake_access_dummy,Outlet_covers_dummy,Stair_gates_dummy,Wide_hallways_dummy,Wide_entrance_for_guests_dummy,Flat_path_to_guest_entrance_dummy,Smart_lock_dummy,Building_staff_dummy,Barbecue_utensils_dummy,Baking_sheet_dummy,Body_soap_dummy,Bath_towel_dummy,Toilet_paper_dummy,Cat_s__dummy,Bedroom_comforts_dummy,Bathroom_essent

# Prep Data

In [7]:
predictors = list(data.columns)
predictors.remove('price')
response = ['price']

X_train = data.loc[:, predictors]
y_train  = np.log(data.loc[:, response])

# # Remove Non ASCII characters 
# X_train.columns = X_train.columns.str.encode('ascii', 'ignore').str.decode('ascii')
# X_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_train.columns]

# GBR

In [9]:
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor()

tuning_parameters = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators' : [250, 500, 750, 1000, 1500, 2000],
    'max_depth' : [2 ,3, 4],
    'subsample' : [0.6, 0.8, 1.0]
}

# Using GridSearchCV would be too slow. Increase the number of iterations to explore more 
# hyperparameter combinations.
gb = RandomizedSearchCV(model, tuning_parameters, n_iter = 32, cv = 5, n_jobs=-1, random_state=87)
gb.fit(X_train, y_train)

print('Best parameters found by randomised search:', gb.best_params_, '\n')

C:\Users\susum\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Best parameters found by randomised search: {'subsample': 0.8, 'n_estimators': 1500, 'max_depth': 4, 'learning_rate': 0.05} 



# LightGBM

In [10]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import lightgbm as lgb

time_start = time.time()

model = lgb.LGBMRegressor(objective='regression')

tuning_parameters = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators' : [250, 500, 750, 1000, 1500, 2000],
    'num_leaves' : list(np.arange(2, 64)),
    'subsample' : [0.6, 0.8, 1.0],
}


lbst = RandomizedSearchCV(model, tuning_parameters, n_iter = 32, cv = 5, n_jobs=-1, random_state=87)
lbst.fit(X_train, y_train)

time_end = time.time()

print(f'time taken is {(time_end - time_start)/60} mins')
print('\nBest parameters found by randomised search:', lbst.best_params_, '\n')

time taken is 1.8524066845575968 mins

Best parameters found by randomised search: {'subsample': 0.8, 'num_leaves': 24, 'n_estimators': 500, 'learning_rate': 0.05} 



# XG Boosting

In [8]:
import xgboost as xgb

time_start = time.time()

model = xgb.XGBRegressor(objective='reg:squarederror')

space = {
    'reg_lambda':  (1, 1000, 'log-uniform'),
    'learning_rate': Real(0.005, 0.1),
    'n_estimators' : Integer(100, 2500),
    'max_depth' : Integer(2, 6),
    'subsample' : Real(0.5, 1.0),
    'colsample_bytree' : Real(0.25, 1.0),
}

np.random.seed(42)
xbst_opt =  BayesSearchCV(model, space, cv = 5,  n_iter=32, scoring = 'neg_mean_squared_error', n_jobs=-1)
xbst_opt.fit(X_train, y_train)
xbst_opt.best_params_

time_end = time.time()

In [9]:
print(f'time taken is {(time_end - time_start)/60} mins')
print('\nBest parameters found by randomised search:', xbst_opt.best_params_, '\n')

time taken is 87.90357467333476 mins

Best parameters found by randomised search: OrderedDict([('colsample_bytree', 0.3064008114322485), ('learning_rate', 0.05329464276006519), ('max_depth', 6), ('n_estimators', 1975), ('reg_lambda', 1000), ('subsample', 1.0)]) 



# Random Forest

In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import GridSearchCV

time_start = time.time()

# Create the parameter grid based on the results of random search 
tuning_parameters = {
    'bootstrap': [True, False],
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
}


# regr = RandomForestRegressor(max_depth=1000, random_state=100)
model = RandomForestRegressor()

# Instantiate the grid search model
rf = RandomizedSearchCV(model, tuning_parameters, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf.fit(X_train, y_train)

time_end = time.time()

print(f'time taken is {(time_end - time_start)/60} mins')

print('\nBest parameters found by randomised search:', rf.best_params_, '\n')

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed: 30.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 85.0min finished
C:\Users\susum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


time taken is 90.54085074663162 mins

Best parameters found by randomised search: {'n_estimators': 1800, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 70, 'bootstrap': True} 

